# Freight Pricing API Example

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
And put your API key in a `SIGNAL_OCEAN_API_KEY` environment variable.

## Calculating a freight price
The Freight pricing API Retrieves freight prices for moving commodities between two port.
The following function is available:

1. get_freight_pricing.

Args:

    vessel_type: The type of vessel to calculate the prices for.
    load_port: Port where the commodity is loaded.
    discharge_port: Port where the commodity is discharged.
    date: Date at which the freight price is requested.
    vessel_subclass: The vessel's subclass.

Returns:

    A collection of freight pricing items, one per vessel class.
    
### Moving commodity from Fujairah towards Rotterdam
The following example will display how to calculate the available freight prices to move a commodity from Fujairah to Rotterdam.
We will use also the API to map ports, vessel classes and vessel types to the Signal Geo schema.

Ports, vessel classes and vessel types can be retrieved through `FreightPricingAPI`:

In [ ]:
from signal_ocean.freight_pricing import FreightPricingAPI

freight_pricing_api = FreightPricingAPI()

freight_pricing_api.get_ports()
freight_pricing_api.get_vessel_classes()
freight_pricing_api.get_vessel_types()


All can be looked up by their name using filters

In [ ]:
from signal_ocean.freight_pricing import FreightPricingAPI, VesselTypeFilter, VesselClassFilter, PortFilter

freight_pricing_api.get_ports(PortFilter(name_like='roa'))
freight_pricing_api.get_vessel_classes(VesselClassFilter(name_like='max'))
freight_pricing_api.get_vessel_types(VesselTypeFilter(name_like='Tank'))

### Call the freight pricing API
Now that we have the ports we want to query about, we can call the freight pricing API and get out the commodity transfer cost.

In [ ]:
from signal_ocean.freight_pricing import FreightPricingAPI,VesselType,VesselSubclass
from datetime import date

freight_pricing_api = FreightPricingAPI()

vessel_type = freight_pricing_api.get_vessel_types(VesselTypeFilter(name_like='tanker'))[0]
load_port = freight_pricing_api.get_ports(PortFilter(name_like='Fujairah'))[0]
discharge_port = freight_pricing_api.get_ports(PortFilter(name_like='Rotterdam'))[0]

today = date.today()

freight_pricing = freight_pricing_api.get_freight_pricing(
    vessel_type,
    load_port=load_port, 
    discharge_port=discharge_port,
    date=today,
    vessel_subclass=VesselSubclass.DIRTY
)

### Print out the result

Now we can hold the result and print it out on a dataframe

In [ ]:
import pandas as pd

dataf = pd.DataFrame([{
    'vessel_class': p.vessel_class,
    'cargo_quantity': p.cargo_quantity,
    **p.costs.__dict__,
    **p.totals.__dict__ } for p in freight_pricing
])

dataf